## Market Basket Analysis (Retail)


In [4]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [3]:
!pip install mlxtend

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 25.4 MB/s eta 0:00:00


In [9]:
data = pd.read_excel('/content/sample_data/Online Retail.xlsx',engine='openpyxl')

In [8]:
!pip install openpyxl

In [10]:
data.shape # get the shape of the data

(541909, 8)

In [11]:
data.head(200) # get the first few transactions in the file

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
195,536388,22469,HEART OF WICKER SMALL,12,2010-12-01 09:59:00,1.65,16250.0,United Kingdom
196,536388,22242,5 HOOK HANGER MAGIC TOADSTOOL,12,2010-12-01 09:59:00,1.65,16250.0,United Kingdom
197,536389,22941,CHRISTMAS LIGHTS 10 REINDEER,6,2010-12-01 10:03:00,8.50,12431.0,Australia
198,536389,21622,VINTAGE UNION JACK CUSHION COVER,8,2010-12-01 10:03:00,4.95,12431.0,Australia


In [12]:
# Clean up spaces in description and remove any rows that don't have a valid invoice
data['Description'] = data['Description'].str.strip()

data.dropna(axis=0, subset=['InvoiceNo'], inplace=True) # remove transactions without an invoice

In [13]:
#remove credit transactions
data['InvoiceNo'] = data['InvoiceNo'].astype('str') # convert the column into a string format
data = data[~data['InvoiceNo'].str.contains('C')] # remove rows that have a 'C' in the InvoiceNo

In [14]:
france_data=data[data['Country'] =="France"] # focus on transactions from france.
france_data.shape

(8408, 8)

In [15]:
# organize the data by invoice and description
france_grouping=france_data.groupby(['InvoiceNo', 'Description'])['Quantity'].sum()
france_grouping

InvoiceNo  Description                    
536370     ALARM CLOCK BAKELIKE GREEN         12
           ALARM CLOCK BAKELIKE PINK          24
           ALARM CLOCK BAKELIKE RED           24
           CHARLOTTE BAG DOLLY GIRL DESIGN    20
           CIRCUS PARADE LUNCH BOX            24
                                              ..
581587     CIRCUS PARADE LUNCH BOX            12
           PACK OF 20 SPACEBOY NAPKINS        12
           PLASTERS IN TIN CIRCUS PARADE      12
           PLASTERS IN TIN STRONGMAN          12
           SPACEBOY LUNCH BOX                 12
Name: Quantity, Length: 8372, dtype: int64

In [16]:
france_matrix=france_grouping.unstack() #expand the grouping into a matrix
france_matrix

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536852,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
537463,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580986,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581001,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581171,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
#convert the matrix into a True/False format: False->not bought, True->bought
france_matrix=france_matrix.notnull()
france_matrix.tail(10)

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
580705,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
580706,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
580736,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
580753,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
580756,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
580986,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581001,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581171,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
581279,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [18]:
#example column distribution
france_matrix['10 COLOUR SPACEBOY PEN'].value_counts()

,count
10 COLOUR SPACEBOY PEN,
False,380
True,12


# Finding frequent itemsets

In [19]:
#Now, let us return the items and itemsets with at least 10% support:
frequent_itemsets=apriori(france_matrix, min_support = 0.1, use_colnames = True)

frequent_itemsets

,support,itemsets
0,0.102041,(ALARM CLOCK BAKELIKE PINK)
1,0.125000,(LUNCH BAG APPLE DESIGN)
2,0.153061,(LUNCH BAG RED RETROSPOT)
3,0.119898,(LUNCH BAG SPACEBOY DESIGN)
4,0.117347,(LUNCH BAG WOODLAND)
5,0.142857,(LUNCH BOX WITH CUTLERY RETROSPOT)
6,0.104592,(MINI PAINT SET VINTAGE)
7,0.102041,(PACK OF 72 RETROSPOT CAKE CASES)
8,0.168367,(PLASTERS IN TIN CIRCUS PARADE)
9,0.137755,(PLASTERS IN TIN SPACEBOY)


# Finding Rules


In [20]:
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5,support_only=False,num_itemsets=france_matrix.shape[0])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(LUNCH BAG APPLE DESIGN),(POSTAGE),0.125000,0.765306,0.104592,0.836735,1.093333,1.0,0.008929,1.437500,0.097561,0.133117,0.304348,0.486701
1,(LUNCH BAG RED RETROSPOT),(POSTAGE),0.153061,0.765306,0.122449,0.800000,1.045333,1.0,0.005310,1.173469,0.051205,0.153846,0.147826,0.480000
2,(LUNCH BAG WOODLAND),(POSTAGE),0.117347,0.765306,0.102041,0.869565,1.136232,1.0,0.012234,1.799320,0.135838,0.130719,0.444234,0.501449
3,(LUNCH BOX WITH CUTLERY RETROSPOT),(POSTAGE),0.142857,0.765306,0.114796,0.803571,1.050000,1.0,0.005466,1.194805,0.055556,0.144695,0.163043,0.476786
4,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.168367,0.170918,0.102041,0.606061,3.545907,1.0,0.073264,2.104592,0.863344,0.430108,0.524848,0.601538
5,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.170918,0.168367,0.102041,0.597015,3.545907,1.0,0.073264,2.063681,0.866000,0.430108,0.515429,0.601538
6,(PLASTERS IN TIN CIRCUS PARADE),(POSTAGE),0.168367,0.765306,0.147959,0.878788,1.148283,1.0,0.019107,1.936224,0.155278,0.188312,0.483531,0.536061
7,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.137755,0.170918,0.104592,0.759259,4.442233,1.0,0.081047,3.443878,0.898687,0.512500,0.709630,0.685600
8,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN SPACEBOY),0.170918,0.137755,0.104592,0.611940,4.442233,1.0,0.081047,2.221939,0.934634,0.512500,0.549943,0.685600
9,(PLASTERS IN TIN SPACEBOY),(POSTAGE),0.137755,0.765306,0.114796,0.833333,1.088889,1.0,0.009371,1.408163,0.094675,0.145631,0.289855,0.491667


In [21]:
!pip install --upgrade mlxtend

# Create a function

In [22]:
# a function to retrieve association rules for a specific country
def get_rules(full_data, country, minimum_support, minimum_confidence):

    country_data=full_data[data['Country'] == country] # focus on transactions from the given country

    # organize the data by invoice and description
    country_grouping=country_data.groupby(['InvoiceNo', 'Description'])['Quantity'].sum()

    country_matrix=country_grouping.unstack() #expand the grouping into a matrix

    #conver the matrix into a True/False format: False->not bought, True->bought
    country_matrix=country_matrix.notnull()

    #get the frequent itemsets
    country_frequent_itemsets=apriori(country_matrix, min_support = minimum_support, use_colnames = True)

    #get the rules
    country_rules = association_rules(country_frequent_itemsets, metric="confidence", min_threshold=minimum_confidence)

    return country_rules

In [23]:
france_rules=get_rules(data, 'France', 0.1, 0.5)
france_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(LUNCH BAG APPLE DESIGN),(POSTAGE),0.125000,0.765306,0.104592,0.836735,1.093333,1.0,0.008929,1.437500,0.097561,0.133117,0.304348,0.486701
1,(LUNCH BAG RED RETROSPOT),(POSTAGE),0.153061,0.765306,0.122449,0.800000,1.045333,1.0,0.005310,1.173469,0.051205,0.153846,0.147826,0.480000
2,(LUNCH BAG WOODLAND),(POSTAGE),0.117347,0.765306,0.102041,0.869565,1.136232,1.0,0.012234,1.799320,0.135838,0.130719,0.444234,0.501449
3,(LUNCH BOX WITH CUTLERY RETROSPOT),(POSTAGE),0.142857,0.765306,0.114796,0.803571,1.050000,1.0,0.005466,1.194805,0.055556,0.144695,0.163043,0.476786
4,(PLASTERS IN TIN CIRCUS PARADE),(PLASTERS IN TIN WOODLAND ANIMALS),0.168367,0.170918,0.102041,0.606061,3.545907,1.0,0.073264,2.104592,0.863344,0.430108,0.524848,0.601538
5,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN CIRCUS PARADE),0.170918,0.168367,0.102041,0.597015,3.545907,1.0,0.073264,2.063681,0.866000,0.430108,0.515429,0.601538
6,(PLASTERS IN TIN CIRCUS PARADE),(POSTAGE),0.168367,0.765306,0.147959,0.878788,1.148283,1.0,0.019107,1.936224,0.155278,0.188312,0.483531,0.536061
7,(PLASTERS IN TIN SPACEBOY),(PLASTERS IN TIN WOODLAND ANIMALS),0.137755,0.170918,0.104592,0.759259,4.442233,1.0,0.081047,3.443878,0.898687,0.512500,0.709630,0.685600
8,(PLASTERS IN TIN WOODLAND ANIMALS),(PLASTERS IN TIN SPACEBOY),0.170918,0.137755,0.104592,0.611940,4.442233,1.0,0.081047,2.221939,0.934634,0.512500,0.549943,0.685600
9,(PLASTERS IN TIN SPACEBOY),(POSTAGE),0.137755,0.765306,0.114796,0.833333,1.088889,1.0,0.009371,1.408163,0.094675,0.145631,0.289855,0.491667


In [24]:
germany_rules=get_rules(data, 'Germany', 0.1, 0.5)
germany_rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(PLASTERS IN TIN CIRCUS PARADE),(POSTAGE),0.115974,0.818381,0.100656,0.867925,1.060539,1.0,0.005746,1.375117,0.064572,0.120735,0.272789,0.495460
1,(PLASTERS IN TIN SPACEBOY),(POSTAGE),0.107221,0.818381,0.100656,0.938776,1.147113,1.0,0.012909,2.966448,0.143649,0.122016,0.662896,0.530885
2,(PLASTERS IN TIN WOODLAND ANIMALS),(POSTAGE),0.137856,0.818381,0.118162,0.857143,1.047364,1.0,0.005344,1.271335,0.052453,0.140992,0.213425,0.500764
3,(REGENCY CAKESTAND 3 TIER),(POSTAGE),0.137856,0.818381,0.120350,0.873016,1.066760,1.0,0.007532,1.430252,0.072589,0.143979,0.300822,0.510037
4,(ROUND SNACK BOXES SET OF 4 FRUITS),(POSTAGE),0.157549,0.818381,0.150985,0.958333,1.171012,1.0,0.022049,4.358862,0.173348,0.183024,0.770582,0.571413
5,(ROUND SNACK BOXES SET OF4 WOODLAND),(POSTAGE),0.245077,0.818381,0.225383,0.919643,1.123735,1.0,0.024817,2.260151,0.145856,0.268930,0.557552,0.597522
6,(WOODLAND CHARLOTTE BAG),(POSTAGE),0.126915,0.818381,0.115974,0.913793,1.116587,1.0,0.012109,2.106783,0.119591,0.139842,0.525343,0.527752
7,(ROUND SNACK BOXES SET OF4 WOODLAND),(ROUND SNACK BOXES SET OF 4 FRUITS),0.245077,0.157549,0.131291,0.535714,3.400298,1.0,0.092679,1.814509,0.935072,0.483871,0.448887,0.684524
8,(ROUND SNACK BOXES SET OF 4 FRUITS),(ROUND SNACK BOXES SET OF4 WOODLAND),0.157549,0.245077,0.131291,0.833333,3.400298,1.0,0.092679,4.529540,0.837922,0.483871,0.779227,0.684524
9,"(POSTAGE, ROUND SNACK BOXES SET OF4 WOODLAND)",(ROUND SNACK BOXES SET OF 4 FRUITS),0.225383,0.157549,0.124726,0.553398,3.512540,1.0,0.089218,1.886357,0.923431,0.483051,0.469878,0.672532


In [25]:
spain_rules=get_rules(data, 'Spain', 0.1, 0.5)
spain_rules

/usr/local/lib/python3.12/dist-packages/pandas/core/nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(6 RIBBONS RUSTIC CHARM),(ASSORTED COLOUR BIRD ORNAMENT),0.166667,0.133333,0.100000,0.600000,4.500000,1.0,0.077778,2.166667,0.933333,0.500000,0.538462,0.675000
1,(ASSORTED COLOUR BIRD ORNAMENT),(6 RIBBONS RUSTIC CHARM),0.133333,0.166667,0.100000,0.750000,4.500000,1.0,0.077778,3.333333,0.897436,0.500000,0.700000,0.675000
2,(6 RIBBONS RUSTIC CHARM),(POSTAGE),0.166667,0.688889,0.144444,0.866667,1.258065,1.0,0.029630,2.333333,0.246154,0.203125,0.571429,0.538172
3,(ASSORTED COLOUR BIRD ORNAMENT),(POSTAGE),0.133333,0.688889,0.122222,0.916667,1.330645,1.0,0.030370,3.733333,0.286713,0.174603,0.732143,0.547043
4,(CLASSIC METAL BIRDCAGE PLANT HOLDER),(POSTAGE),0.111111,0.688889,0.100000,0.900000,1.306452,1.0,0.023457,3.111111,0.263889,0.142857,0.678571,0.522581
5,(PACK OF 72 RETROSPOT CAKE CASES),(POSTAGE),0.122222,0.688889,0.122222,1.000000,1.451613,1.0,0.038025,inf,0.354430,0.177419,1.000000,0.588710
6,(REGENCY CAKESTAND 3 TIER),(POSTAGE),0.244444,0.688889,0.188889,0.772727,1.121701,1.0,0.020494,1.368889,0.143599,0.253731,0.269481,0.523460
7,"(POSTAGE, 6 RIBBONS RUSTIC CHARM)",(ASSORTED COLOUR BIRD ORNAMENT),0.144444,0.133333,0.100000,0.692308,5.192308,1.0,0.080741,2.816667,0.943723,0.562500,0.644970,0.721154
8,"(POSTAGE, ASSORTED COLOUR BIRD ORNAMENT)",(6 RIBBONS RUSTIC CHARM),0.122222,0.166667,0.100000,0.818182,4.909091,1.0,0.079630,4.583333,0.907173,0.529412,0.781818,0.709091
9,"(6 RIBBONS RUSTIC CHARM, ASSORTED COLOUR BIRD ...",(POSTAGE),0.100000,0.688889,0.100000,1.000000,1.451613,1.0,0.031111,inf,0.345679,0.145161,1.000000,0.572581
